In [1]:
from pylab import *
import pyresample as pr
import time
import gc
import matplotlib.colors as colors
from mpl_utils import *
from mpl_toolkits.axes_grid1 import make_axes_locatable
import sys
import numpy.ma as ma
import matplotlib.pyplot as plt

from mwir_sst_v4 import MWIRSST

In [2]:
def gmtColormap(fileName, GMTPath=None):
  import colorsys
  import numpy
  N = numpy
  if type(GMTPath) == type(None):
      filePath = "/usr/local/cmaps/"+ fileName+".cpt"
  else:
      filePath = GMTPath+"/"+ fileName +".cpt"
  try:
      f = open(filePath)
  except:
      print "file ",filePath, "not found"
      return None

  lines = f.readlines()
  f.close()

  x = []
  r = []
  g = []
  b = []
  colorModel = "RGB"
  for l in lines:
      ls = l.split()
      if l[0] == "#":
         if ls[-1] == "HSV":
             colorModel = "HSV"
             continue
         else:
             continue
      if ls[0] == "B" or ls[0] == "F" or ls[0] == "N":
         pass
      else:
          x.append(float(ls[0]))
          r.append(float(ls[1]))
          g.append(float(ls[2]))
          b.append(float(ls[3]))
          xtemp = float(ls[4])
          rtemp = float(ls[5])
          gtemp = float(ls[6])
          btemp = float(ls[7])

  x.append(xtemp)
  r.append(rtemp)
  g.append(gtemp)
  b.append(btemp)

  nTable = len(r)
  x = N.array( x , N.float)
  r = N.array( r , N.float)
  g = N.array( g , N.float)
  b = N.array( b , N.float)
  if colorModel == "HSV":
     for i in range(r.shape[0]):
         rr,gg,bb = colorsys.hsv_to_rgb(r[i]/360.,g[i],b[i])
         r[i] = rr ; g[i] = gg ; b[i] = bb
  if colorModel == "HSV":
     for i in range(r.shape[0]):
         rr,gg,bb = colorsys.hsv_to_rgb(r[i]/360.,g[i],b[i])
         r[i] = rr ; g[i] = gg ; b[i] = bb
  if colorModel == "RGB":
      r = r/255.
      g = g/255.
      b = b/255.
  xNorm = (x - x[0])/(x[-1] - x[0])

  red = []
  blue = []
  green = []
  for i in range(len(x)):
      red.append([xNorm[i],r[i],r[i]])
      green.append([xNorm[i],g[i],g[i]])
      blue.append([xNorm[i],b[i],b[i]])
  colorDict = {"red":red, "green":green, "blue":blue}
  return (colorDict)

In [3]:
def setColorbar(gtmName):

    cm = gmtColormap(fileName=gtmName, GMTPath='/var/calculate/prj/posada/handlers/gmtcolorbar')
    return cm

In [4]:
def read_data(filename='mw_ir.fusion.2015.353.rt.gz'):
    dataset = MWIRSST(filename, missing=missing)
    if not dataset.variables: sys.exit('problem reading file')
    return dataset

ilon = (769,774)
ilat = (473,477)
avar = 'sst'
missing = -999.

   
dataset = read_data()
print
print 'done'

lons = dataset.variables['longitude'][:]
lats = dataset.variables['latitude'][:]

x,y = np.meshgrid(lons,lats)

grid_def = pr.geometry.GridDefinition(x, y)

product_parameters = {}
product_parameters['sst'] = {'palette': cm.jet, 'range_min': 0, 'range_max': 30}

ice_palette_dict = setColorbar('GMT_ice_con')
ice_palette = colors.LinearSegmentedColormap('liquid', ice_palette_dict, 256)

product_parameters['ice'] = {'palette': ice_palette, 'range_min': 0.0, 'range_max': 1.0}

projections = [{'name': 'epsg_4326', 'inch_size': (20.48, 10.24)},
               {'name': 'epsg_3413', 'inch_size': (20.48, 20.48)}, #north
               {'name': 'epsg_3031', 'inch_size': (20.48, 20.48)}] #south



done
file  /var/calculate/prj/posada/handlers/gmtcolorbar/GMT_ice_con.cpt not found


/usr/local/lib/python2.7/dist-packages/pyresample/geometry.py:76: UserWarning: All geometry objects expect longitudes in the [-180:+180[ range. We will now automatically wrap your longitudes into [-180:+180[, and continue. To avoid this warning next time, use routine utils.wrap_longitudes().
  'To avoid this warning next time, use routine utils.wrap_longitudes().')


In [5]:
import os
def mkdir_p(path_dir):
    try:
        os.makedirs(path_dir)
    except:
        pass

In [6]:
variables_list = ['sst', 'ice']
fill_value = dataset.missing
for pp_name in variables_list:
    if not pp_name in product_parameters:
        continue
    print "\nStart parameter: ", pp_name
    pp_value = dataset.variables[pp_name][:]
#     print pp_value.shape
#     pp_value = resize(pp_value, (pp_value.shape[1], pp_value.shape[2]))
    valid_min = dataset.variables[pp_name].valid_min
    valid_max = dataset.variables[pp_name].valid_max
    
    pp_value = ma.masked_where(pp_value > valid_max, pp_value)
    pp_value = ma.masked_where(pp_value < valid_min, pp_value)
    pp_value = np.ma.masked_equal(pp_value, dataset.missing)
    
    for projection in projections:
        directory_to_save = "/tmp/mwirsst/test/%s" %(projection['name'])
        area_def = pr.utils.load_area('/var/calculate/prj/posada/handlers/resources/areas.cfg', '%s_crude'%(projection['name']))

        result = pr.kd_tree.resample_nearest(grid_def, pp_value.ravel(), area_def, radius_of_influence=30000,
                             fill_value=dataset.missing, nprocs=2)
    
        #specify figure size in inches:
        fig = figure(num=1, figsize=(projection['inch_size']), facecolor=None, edgecolor=None,
                     frameon=False)
        fig.set_size_inches(projection['inch_size'][0], projection['inch_size'][1])
        ax = axes([0., 0., 1., 1.])
        ax.set_axis_off()
        fig.add_axes(ax)
        m = pr.plot.area_def2basemap(area_def, resolution=None)
        
        palette = product_parameters[pp_name]['palette']
        pal_norm = colors.Normalize(vmin=product_parameters[pp_name]['range_min'],
                                    vmax=product_parameters[pp_name]['range_max'], clip=True)
        
        m.imshow(result, origin='upper', cmap=palette, norm=pal_norm)
        
        mkdir_p(directory_to_save)
        savepath = '%s/%s.png'%(directory_to_save, pp_name)
        savefig(savepath, dpi=100, transparent=True, pad_inches=0)
        del fig
    del pp_value
del dataset
gc.collect()


Start parameter:  sst


IOError: [Errno 2] No such file or directory: '/var/calculate/prj/posada/handlers/resources/areas.cfg'